In [1]:
def factorial(integer):
    """ Returns factorial of the given integer"""
    n = int(integer)
    if n<=1:
        return 1
    fact=1
    for i in range(1, n+1):
        fact*=i
    return fact

In [2]:
import gradio
gradio.Interface(factorial, inputs="text", outputs="text").launch(share=True)

Running on local URL:  http://127.0.0.1:7860/
Running on public URL: https://30609.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x14e64c9f0d0>,
 'http://127.0.0.1:7860/',
 'https://30609.gradio.app')

In [ ]:
import gradio as gr
import tensorflow as tf
import numpy as np
import requests
 
# loading the model
inception_net = tf.keras.applications.InceptionV3()
 
# Download human-readable labels.
response = requests.get("https://git.io/JJkYN")
labels = response.text.split("\n")
 
def classify_image(image):
    """ Returns a dictionary with key as label and values
    as the predicted confidence for that label"""
    # reshaping the image
    image = image.reshape((-1, 299, 299, 3))
    # preprocessing the image for inception_v3
    image = tf.keras.applications.inception_v3.preprocess_input(image)
    # predicting the output
    prediction = inception_net.predict(image).flatten()
    return {labels[i]: float(prediction[i]) for i in range(1000)}
 
# initializing the input component
image = gr.inputs.Image(shape = (299, 299, 3))
# initializing the output component
label = gr.outputs.Label(num_top_classes = 3)
 
# launching the interface
gr.Interface(fn = classify_image, inputs = image,
             outputs = label, capture_session = True).launch()

In [6]:
import pytesseract
import cv2
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

image = cv2.imread("./repository-open-graph-template.png")

text = pytesseract.image_to_data(image, output_type='data.frame')

text = text[text.conf != -1]
lines = text.groupby('block_num')['text'].apply(list)

conf = text.groupby(['block_num'])['conf'].mean()



block_num
1                                         [C), GitHub]
2                               [Repo, Card, Template]
3    [We, recommend, leaving, a, 40pt, border, arou...
4                                                  [ ]
Name: text, dtype: object

In [ ]:
import gradio as gr
import pandas as pd
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def ocr(image):
    text = pytesseract.image_to_data(image, output_type='data.frame')

    text = text[text.conf != -1]
    lines = text.groupby(['page_num', 'block_num', 'par_num', 'line_num'])['text'] \
                                     .apply(lambda x: ' '.join(list(x))).tolist()
    confs = text.groupby(['page_num', 'block_num', 'par_num', 'line_num'])['conf'].mean().tolist()
        
    line_conf = []
        
    for i in range(len(lines)):
        if lines[i].strip():
            line_conf.append((lines[i], round(confs[i],3)))
    
    labels = [line_conf[i][0] for i in range(len(line_conf))]
    confs = [line_conf[i][1] for i in range(len(line_conf))]

    data = {'label': labels, 'confidence': confs}

    df = pd.DataFrame(data)

    return {labels[i]: float(confs[i]/100) for i in range(len(labels))}

image = gr.inputs.Image()
label = gr.outputs.Label()

gr.Interface(fn = ocr, inputs = image,outputs = label).launch(share=True)